In [1]:
import numpy as np
import os  # for mkdir
import sys
import math
import librosa # conda install -c conda-forge librosa
import soundfile as sf

##-----------   paramManager  interface   ------------------##
# git clone https://github.com/prashanthtr/paramManager
from parammanager import paramManager
from sonyganformat import sonyGanJson

### Chunk file into 1 seconds and based on SampleRate

In [2]:
chunkDuration = 1; # 1 sec
resampFreq = 22050;

In [3]:
from utils import longSilence

datapath='srcwav'  #root folder
fileList = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f)) and (os.path.splitext(f)[1]==".wav" or os.path.splitext(f)[1]==".aif" or os.path.splitext(f)[1]==".aiff")]
#stub: fileList = ["silence.wav"]


createSegments = True

if createSegments == True:
    for fileName in fileList:

        sig,sr=librosa.core.load(datapath + "/" + fileName, sr=None)
        
        # resample
        sig= librosa.core.resample(sig, sr, resampFreq) 

        fileBase = os.path.splitext(fileName)[0]

        numSamplesPerChunk = int(chunkDuration*resampFreq);
        numChunks = math.floor(len(sig)/numSamplesPerChunk)
        segment = []

        mean = np.mean(sig)

        for i in range(numChunks-1):
            segment = sig[i*numSamplesPerChunk:(i+1)*numSamplesPerChunk]
            mean = np.mean(segment)
            x = [sample-mean for sample in segment] #dc filter
            if( longSilence(x, 0.01) ):
                print(fileBase + " ",  i)
                norm = librosa.util.normalize(x)
                sf.write("segmented" + "/" + fileBase + "--" + "v-" + str(i) + ".wav", norm, resampFreq)
            else:
                print("Discarding a silent segment ", i)
            i = i + 1;

arp_1  0
arp_1  1
arp_1  2
arp_1  3
arp_1  4
arp_1  5
arp_1  6
arp_1  7
arp_1  8
arp_1  9
arp_1  10
arp_1  11
arp_1  12
arp_1  13
arp_1  14
arp_1  15
arp_1  16
arp_1  17
arp_1  18
arp_1  19
arp_1  20
arp_1  21
arp_1  22
arp_1  23
arp_1  24
arp_1  25
arp_1  26
arp_1  27
arp_1  28
arp_1  29
arp_1  30
arp_1  31
arp_1  32
arp_1  33
arp_1  34
arp_1  35
arp_1  36
arp_1  37
arp_1  38
arp_1  39
arp_1  40
arp_1  41
arp_1  42
arp_1  43
arp_1  44
arp_1  45
arp_1  46
arp_1  47
arp_1  48
arp_1  49
arp_1  50
arp_1  51
arp_1  52
arp_1  53
arp_1  54
arp_1  55
arp_1  56
arp_1  57
arp_1  58
arp_1  59
arp_1  60
arp_1  61
arp_1  62
arp_1  63
arp_1  64
arp_1  65
arp_1  66
arp_1  67
arp_1  68
arp_1  69
arp_1  70
arp_1  71
arp_1  72
arp_1  73
arp_1  74
arp_1  75
arp_1  76
arp_1  77
arp_1  78
arp_1  79
arp_1  80
arp_1  81
arp_1  82
arp_1  83
arp_1  84
arp_1  85
arp_1  86
arp_1  87
arp_1  88
arp_1  89
arp_1  90
arp_1  91
arp_1  92
arp_1  93
arp_1  94
arp_1  95
arp_1  96
arp_1  97
arp_1  98
arp_1  99
arp_1  100

### Generate record formats (parammanager, sonyGan, tfrecords)

In [4]:
#Setup paths for paramManager and examplesJSON
datapath = 'segmented'
parampath = 'segmented'

### ParamManager

In [5]:
pm=paramManager.paramManager(datapath,parampath) #filebase is directory name
pm.initParamFiles(overwrite=True) ##-----------   paramManager  interface ------------------##
pm.checkIntegrity()

paramList = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(parampath, os.path.splitext(f)[0] + ".params"))]
for p in paramList:
    soundText = os.path.splitext(p)[0].split("--")
    foo=pm.getParams(datapath + '/' + p) #extension is optional
    pm.addMetaParam(parampath + '/' + p,"soundName", soundText[0])
    pm.addMetaParam(parampath + '/' + p,"segmentNum", int(soundText[len(soundText)-1].split("-")[1]))

In [6]:
paramList = ["arp_1--v-123.params"]

for p in paramList:
    foo=pm.getParams(parampath + '/' + p) #extension is optional
    print(foo)

{'meta': {'filename': 'arp_1--v-123.wav', 'soundName': 'arp_1', 'segmentNum': 123}}


### SonyGan format

In [7]:
# Read all files in datapath and create a examples.json
print("parameters")
sg = sonyGanJson.SonyGanJson(datapath,"natural",22050,"O'REILLY")

parameters
rate
irreg
cf
v
unrecognized param


### Writing records in single SG json file

In [8]:
#sg.ppRecords() # pretty prints in json format
sg.write2File("sonyGan.json") 

written output to  sonyGan.json
